In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix


import re
import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-dataset/training.1600000.processed.noemoticon.csv
/kaggle/input/sentiment-analysis-dataset/train.csv
/kaggle/input/sentiment-analysis-dataset/testdata.manual.2009.06.14.csv
/kaggle/input/sentiment-analysis-dataset/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/sentiment-analysis-dataset/training.1600000.processed.noemoticon.csv',
                 delimiter=',', encoding='ISO-8859-1')
df.columns = ['Sentiment','id','date','query','user','text']
df.head()

,Sentiment,id,date,query,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [3]:
df = df[['Sentiment','text']]
df['Sentiment'] = df['Sentiment'].replace({0: 'negative', 4: 'positive'})


In [4]:
df.Sentiment.value_counts()

Sentiment
negative    799996
positive    248576
Name: count, dtype: int64

In [5]:
from sklearn.utils import resample

df_positive = df[df['Sentiment'] == 'positive']
df_negative = df[df['Sentiment'] == 'negative']

# Downsample majority class
df_neg_downsampled = resample(df_negative, 
                                   replace=False,
                                   n_samples=len(df_positive),
                                   random_state=42)

# Combine downsampled majority class with minority class
df_balanced = pd.concat([df_neg_downsampled, df_positive])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [6]:
df = df_balanced
df['Sentiment'] = df['Sentiment'].replace({'negative': 0, 'positive': 1})
X = df['text']
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/tmp/ipykernel_36/1639195524.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sentiment'] = df['Sentiment'].replace({'negative': 0, 'positive': 1})


In [7]:
df.head

<bound method NDFrame.head of         Sentiment                                               text
0               1  @firsttiger Real phone? i just read your blog ...
1               0  is not allowed 2 see her Babyy during  finals....
2               1  @nicolerichie absolutely, my sister used to pr...
3               0                 I want holidays!!! Tired of exams 
4               0  @HyunINC I'll be there Sunday night up until T...
...           ...                                                ...
497147          1                 Another amusing tweeter: @tinafey 
497148          1             @MichaelMCrowley i saw wolverine too! 
497149          0              Losing my voice  http://myloc.me/2gEs
497150          0  Just settled down to watch the football, then ...
497151          0  Lookin for french articles for my oral exam  h...

[497152 rows x 2 columns]>

# *Naive Bayes* - CountVectorizer

In [8]:
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

#Create and train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train)

#Make predictions on the test set
y_pred = clf.predict(X_test_vectorized)

# 7. Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[40907  8568]
 [13047 36909]]
              precision    recall  f1-score   support

           0       0.76      0.83      0.79     49475
           1       0.81      0.74      0.77     49956

    accuracy                           0.78     99431
   macro avg       0.78      0.78      0.78     99431
weighted avg       0.79      0.78      0.78     99431



# *Naive Bayes* - TFIDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

#Make predictions
y_pred = clf.predict(X_test_tfidf)

#Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[38865 10610]
 [12193 37763]]
              precision    recall  f1-score   support

           0       0.76      0.79      0.77     49475
           1       0.78      0.76      0.77     49956

    accuracy                           0.77     99431
   macro avg       0.77      0.77      0.77     99431
weighted avg       0.77      0.77      0.77     99431



# *Logistic Regression*

In [10]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42)
clf.fit(X_train_vectorized, y_train)

#Make predictions on the test set
y_pred = clf.predict(X_test_vectorized)

# 7. Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[38560 10915]
 [ 9631 40325]]
              precision    recall  f1-score   support

           0       0.80      0.78      0.79     49475
           1       0.79      0.81      0.80     49956

    accuracy                           0.79     99431
   macro avg       0.79      0.79      0.79     99431
weighted avg       0.79      0.79      0.79     99431



# *Logistic Regression* - TFIDF

In [11]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train_tfidf, y_train)

#Make predictions on the test set
y_pred = clf.predict(X_test_tfidf)

# 7. Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[38653 10822]
 [10049 39907]]
              precision    recall  f1-score   support

           0       0.79      0.78      0.79     49475
           1       0.79      0.80      0.79     49956

    accuracy                           0.79     99431
   macro avg       0.79      0.79      0.79     99431
weighted avg       0.79      0.79      0.79     99431



# *Gradient Boost*

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)

# 7. Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[30818 18657]
 [10948 39008]]
              precision    recall  f1-score   support

           0       0.74      0.62      0.68     49475
           1       0.68      0.78      0.72     49956

    accuracy                           0.70     99431
   macro avg       0.71      0.70      0.70     99431
weighted avg       0.71      0.70      0.70     99431



# *MLP*

In [13]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)

# 7. Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[[39662  9813]
 [11622 38334]]
              precision    recall  f1-score   support

           0       0.77      0.80      0.79     49475
           1       0.80      0.77      0.78     49956

    accuracy                           0.78     99431
   macro avg       0.78      0.78      0.78     99431
weighted avg       0.78      0.78      0.78     99431



# *Xgboost*

In [14]:
from xgboost import XGBClassifier

clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
clf.fit(X_train_tfidf, y_train)

# 7. Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[39662  9813]
 [11622 38334]]
              precision    recall  f1-score   support

           0       0.77      0.80      0.79     49475
           1       0.80      0.77      0.78     49956

    accuracy                           0.78     99431
   macro avg       0.78      0.78      0.78     99431
weighted avg       0.78      0.78      0.78     99431



# *distilbert-base-uncased*

In [15]:
from transformers import pipeline
sentiment_task = pipeline("sentiment-analysis")
y_pred = sentiment_task(X_test.tolist()[:5000])


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
y_pred = pd.DataFrame(y_pred)
y_pred = y_pred['label']
y_pred = y_pred.replace({'POSITIVE': 1, 'NEGATIVE': 0})

print(confusion_matrix(y_test[:5000], y_pred))
print(classification_report(y_test[:5000], y_pred))

[[2017  491]
 [ 975 1517]]
              precision    recall  f1-score   support

           0       0.67      0.80      0.73      2508
           1       0.76      0.61      0.67      2492

    accuracy                           0.71      5000
   macro avg       0.71      0.71      0.70      5000
weighted avg       0.71      0.71      0.70      5000



/tmp/ipykernel_36/466298412.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_pred = y_pred.replace({'POSITIVE': 1, 'NEGATIVE': 0})


# *roberta-base*

In [18]:
from transformers import pipeline
sentiment_task = pipeline("sentiment-analysis", model = 'roberta-base')
y_pred = sentiment_task(X_test.tolist()[:5000])

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
y_pred = pd.DataFrame(y_pred)
y_pred = y_pred['label']

y_pred = y_pred.replace({'LABEL_1': 1, 'LABEL_0': 0})

print(confusion_matrix(y_test[:5000], y_pred))
print(classification_report(y_test[:5000], y_pred))


[[2508    0]
 [2492    0]]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67      2508
           1       0.00      0.00      0.00      2492

    accuracy                           0.50      5000
   macro avg       0.25      0.50      0.33      5000
weighted avg       0.25      0.50      0.34      5000



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# *deberta-v3-base-absa-v1.1*

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the ABSA model and tokenizer
model_name = "yangheng/deberta-v3-base-absa-v1.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("text-classification", model=model_name, tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [26]:
y_pred = classifier(X_test.tolist()[:200])

In [27]:
y_pred = pd.DataFrame(y_pred)
y_pred
y_pred = y_pred['label']

y_pred = y_pred.replace({'Positive': 1, 'Negative': 0, 'Neutral':2})
print(confusion_matrix(y_test[:200], y_pred))
print(classification_report(y_test[:200], y_pred))

[[48 27 24]
 [ 7 63 31]
 [ 0  0  0]]
              precision    recall  f1-score   support

           0       0.87      0.48      0.62        99
           1       0.70      0.62      0.66       101
           2       0.00      0.00      0.00         0

    accuracy                           0.56       200
   macro avg       0.52      0.37      0.43       200
weighted avg       0.79      0.56      0.64       200



/tmp/ipykernel_36/2948050310.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_pred = y_pred.replace({'Positive': 1, 'Negative': 0, 'Neutral':2})
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda